In [1]:
import os
from os.path import join
import urllib.request #https://stackoverflow.com/questions/17960942/attributeerror-module-object-has-no-attribute-urlretrieve
from html_parser import retrieve_info_for_galaxy

In [2]:
#constants:
PATH_TO_GALAXY_INFO_FILE = "..\\table_info\\size_of_galaxies\\Table3_part3.txt"
DOWNLOAD_PATH = ""

In [3]:
size = """NGC598: 6000
NGC5236: 2000"""

In [4]:
#helper functions:
def get_names_and_pixel_size(file_path, delimeter=":"):
    name_to_pixel_dict = dict()
    for line in size.strip().split("\n"):
        name = line.split(delimeter)[0].strip()
        pixel_size = line.split(delimeter)[1].strip()
        name_to_pixel_dict[name] = pixel_size
    """
    with open(file_path,'r') as f:
        for each_line in f.readlines():
            if not each_line.startswith('#') and delimeter in each_line:
                name = each_line.split(delimeter)[0].strip()
                pixel_size = each_line.split(delimeter)[1].strip()
                name_to_pixel_dict[name] = pixel_size
    """
    return name_to_pixel_dict

def write_to_file(to_write,folder,file_name):
    check_if_folder_exists_and_if_not_make_folder(folder)
    path = join(folder,file_name)
    
    with open(path, 'a') as out:
        out.write(to_write + '\n')
    out.close()

def check_if_folder_exists_and_if_not_make_folder(folder):
    '''create folder if does not exist'''
    if not os.path.exists(folder):
        os.makedirs(folder)
        
def download_fits_file(url_to_fit,name,band,is_mask=False):
    folder = join(DOWNLOAD_PATH,name)
    check_if_folder_exists_and_if_not_make_folder(folder)

    file_name_ending = "_mask" if is_mask else ""

    file_name = "{}_{}{}.fits".format(name,band,file_name_ending)
    path_to_save = join(folder,file_name)
    urllib.request.urlretrieve(url_to_fit,path_to_save)

def download_color(url_to_color, name):
    folder = join(DOWNLOAD_PATH,name)

    file_name = "{}_color.jfif".format(name)
    path_to_save = join(folder,file_name)
    urllib.request.urlretrieve(url_to_color,path_to_save)

In [5]:
#primary function:
def run_on_galaxy(name,pixel_size=240):
    (url, ra_dec_string, arcsec_size_string, color_image_link, fits_cutout_link_dict, fits_cutout_mask_link_dict) = retrieve_info_for_galaxy(name,pixel_size)
    folder = join(DOWNLOAD_PATH,name)
    
    #print(folder)
    
    if url != "":
        write_to_file(url,folder,"{}_link.txt".format(name))
    if ra_dec_string != "":
        write_to_file(ra_dec_string,folder,"{}_ra_dec.txt".format(name))
    if arcsec_size_string != "":
        size_string = "{} pixels {} arcsec".format(pixel_size,arcsec_size_string)
        write_to_file(size_string,folder,"{}_size.txt".format(name))
    if len(fits_cutout_link_dict) != 0:
        for each_band in fits_cutout_link_dict:
            url_to_fit = fits_cutout_link_dict[each_band]
            download_fits_file(url_to_fit,name,each_band,is_mask=False)
    if len(fits_cutout_link_dict) != 0:
        for each_band in fits_cutout_mask_link_dict:
            url_to_fit = fits_cutout_mask_link_dict[each_band]
            download_fits_file(url_to_fit,name,each_band,is_mask=True)
    if color_image_link != "":
        download_color(color_image_link, name)

In [6]:
#run with galaxy:

gals = get_names_and_pixel_size(PATH_TO_GALAXY_INFO_FILE)
total_gals = len(gals); current = 1
print(gals)


for each_gal in gals:
    the_size = gals[each_gal]
    
    print("{}: {} (#{} of {})".format(each_gal, the_size, current, total_gals))
    
    try:
        run_on_galaxy(each_gal,the_size)
        #break
    except Exception as e:
        print("Error running on galaxy: {}".format(each_gal))
        print(e)
        
    current += 1
        

    

{'NGC1232': '1200', 'NGC2553': '800', 'NGC3344': '800', 'NGC3346': '800', 'NGC3351': '800', 'NGC3359': '1000', 'NGC3367': '600', 'NGC3381': '600', 'NGC3395': '400', 'NGC3423': '800', 'NGC3445': '400', 'PGC39728': '200', 'PGC46767': '400', 'PGC49906': '200'}
NGC1232: 1200 (#1 of 14)
NGC2553: 800 (#2 of 14)
NGC3344: 800 (#3 of 14)
NGC3346: 800 (#4 of 14)
NGC3351: 800 (#5 of 14)
NGC3359: 1000 (#6 of 14)
NGC3367: 600 (#7 of 14)
NGC3381: 600 (#8 of 14)
NGC3395: 400 (#9 of 14)
NGC3423: 800 (#10 of 14)
NGC3445: 400 (#11 of 14)
PGC39728: 200 (#12 of 14)
PGC46767: 400 (#13 of 14)
PGC49906: 200 (#14 of 14)
